<a href="https://colab.research.google.com/github/tayfununal/solving_hamiltonian_with_deeplearning/blob/main/stage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

from scipy import integrate

plt.style.use('seaborn-poster')
sns.set_style({'font.family':'serif', 'font.serif':'Times New Roman'})

In [ ]:
%run hamiltonian/symplectic_euler.ipynb
%run hamiltonian/verlet.ipynb
%run hamiltonian/hamiltonian.ipynb

%run hamiltonian/non-linear_oscillator_odeint.ipynb
%run hamiltonian/Lotka-Volterra_odeint.ipynb

In [ ]:
tf.random.set_seed(12345)
np.random.seed(12345)
random.seed(12345)

In [ ]:
# --------------------------------------------------Harmonic Oscillator Example----------------------------------------------------
def ho_T(p):
    return (p**2)/2

def ho_V(q):
    return (q**2)/2

ho_model = Hamiltonian(solver=verlet(ho_T, ho_V, initial_point=[0.,1.], t_interval=[0, 2*np.pi], h=5), input_dim=1, output_dim=2,
            hidden_layer=20, each_neuron=64, activation_func=tf.math.sin, epoch=2000 ,batch_size=100000000, learning_rate=0.0001, h_for_t_created=1000)

In [ ]:
plt.figure(figsize = (5.5, 4))
plt.xlabel('q', fontsize=17)
plt.ylabel('p', fontsize=17)

ho_test_t = np.sort(np.random.uniform(0,2*np.pi,1000))
ho_prediction_energy = ho_model.energy_model_train(ho_test_t)
ho_prediction_mse = ho_model.model_train(ho_test_t)

# grand-truth
plt.scatter(tf.math.sin(np.linspace(0.,2*np.pi,10)), tf.math.cos(np.linspace(0.,2*np.pi,10)), c="green", alpha=0.6, label='grand-truth')
# symplectic euler
plt.plot(ho_model.z[:,0], ho_model.z[:,1], c="blue", alpha=0.6, label='Verlet')
# NN with mse
plt.plot(ho_prediction_mse[:,0], ho_prediction_mse[:,1], c="orange", alpha=0.6, label='Model with MSE loss')
# NN with energy loss
plt.plot(ho_prediction_energy[:,0], ho_prediction_energy[:,1], c="red", alpha=0.6, label='Model with energy\nconserved loss')
plt.legend(loc='upper right', bbox_to_anchor=(1.8, 1.036))

In [ ]:
plt.figure(figsize = (4, 4))
plt.xlabel('t', fontsize=17)
plt.ylabel('Energy', fontsize=17)

ho_energy = [ho_model.H(ho_T, ho_V, 1.0, 0.0) for i in np.linspace(0.,2*np.pi,10)]
ho_energy_new = ho_model.H(ho_T, ho_V, ho_prediction_energy[:,0], ho_prediction_energy[:,1])
ho_symplectic_new = ho_model.H(ho_T, ho_V, ho_model.z[:,0], ho_model.z[:,1])

plt.plot(np.linspace(0.,2*np.pi,10), ho_energy,'o', c='green', alpha=0.6, label='expected energy')
plt.plot(ho_model.t, ho_symplectic_new, alpha=0.6, c="blue", label='Verlet energy')
plt.plot(ho_test_t, ho_energy_new, alpha=0.6, c="red", label='Model energy')
plt.legend(loc='upper right', bbox_to_anchor=(1.97, 1.036))

In [ ]:
tf.random.set_seed(12345)
np.random.seed(12345)
random.seed(12345)

In [ ]:
# --------------------------------------------------Non-Linear Oscillator Example----------------------------------------------------

def non_os_T(p):
  return (p**2)/2

def non_os_V(q):
  return ((q**2)/2) + ((q**4)/4)

non_os_model = Hamiltonian(solver=verlet(non_os_T, non_os_V, initial_point=[1.3, 1.0], t_interval=[0., 4*np.math.pi], h=20), input_dim=1, output_dim=2,
              hidden_layer=20, each_neuron=64, activation_func=tf.math.sin, epoch=2000, 
              batch_size=1000000000000, learning_rate=0.0001, h_for_t_created=1000)

In [ ]:
plt.figure(figsize = (4, 4))
plt.xlabel('q', fontsize=17)
plt.ylabel('p', fontsize=17)

non_os_test_t = np.sort(np.random.uniform(0., 4*np.math.pi, 1000))
non_os_prediction_energy = non_os_model.energy_model_train(non_os_test_t)
non_os_prediction_mse = non_os_model.model_train(non_os_test_t)

# grand-truth
non_os_odeint_result = non_os_odeint(np.array([1.3,1.0]), t=np.linspace(0., 4*np.pi, 1500))
plt.plot(non_os_odeint_result[:,0], non_os_odeint_result[:,1], 'o',c="green", alpha=0.6, label= 'grand-truth')

# symplectic euler
plt.plot(non_os_model.z[:,0], non_os_model.z[:,1], c ="blue", alpha=0.6, label='Verlet')
# NN with mse
plt.plot(non_os_prediction_mse[:,0], non_os_prediction_mse[:,1], c ="orange", alpha=0.6, label='Model with MSE loss')
# NN with energy loss
plt.plot(non_os_prediction_energy[:,0], non_os_prediction_energy[:,1], c="red", alpha=0.6, label='Model with energy\nconserved loss')
plt.legend(loc='upper right', bbox_to_anchor=(2.05,1.036))

In [ ]:
plt.figure(figsize = (4, 4))
plt.xlabel('t', fontsize=17)
plt.ylabel('Energy', fontsize=17)

non_energy = [non_os_model.H(non_os_T, non_os_V, 1.3, 1.0) for i in np.linspace(0., 4 * np.pi, 20)]
non_energy_new = non_os_model.H(non_os_T, non_os_V, non_os_prediction_energy[:,0], non_os_prediction_energy[:,1])
non_symplectic_new = non_os_model.H(non_os_T, non_os_V, non_os_model.z[:,0], non_os_model.z[:,1])

plt.plot(np.linspace(0., 4 * np.pi, 20), non_energy, 'o', c='green', alpha=0.6, label='expected energy')
plt.plot(non_os_model.t, non_symplectic_new, alpha=0.6, c="blue",label='Verlet energy')
plt.plot(non_os_test_t, non_energy_new, alpha=0.6, c="red", label='Model energy')
plt.legend(loc='upper right', bbox_to_anchor=(1.93, 1.036))

In [ ]:
tf.random.set_seed(12345)
np.random.seed(12345)
random.seed(12345)

In [ ]:
# --------------------------------------------------Lotka-Volterra Example----------------------------------------------------

# α = 2/3, β = 4/3, γ = 1 = δ. Assume x, y quantify thousands each. Circles represent prey and predator initial conditions from x = y = 0.9 to 1.8, in steps of 0.1. The fixed point is at (1, 1/2).
# q = ln(y), p = ln(x) 

def lv_T(p):
  return tf.math.exp(p) - p

def lv_V(q):
  return (4/3) * tf.math.exp(q) - (2/3) * q

lv_model = Hamiltonian(solver=verlet(lv_T, lv_V, initial_point=[-0.105, -0.105], t_interval=[0, 50], h=30), input_dim=1, output_dim=2,
            hidden_layer=20, each_neuron=64, activation_func=tf.math.sin, epoch=2000 ,batch_size=100000000, learning_rate=0.0001, h_for_t_created=1000)

In [ ]:
plt.figure(figsize = (4, 4))
plt.xlabel('x', fontsize=17)
plt.ylabel('y', fontsize=17)
#plt.title("Phase Diagram", fontsize=17)

lv_test_t = np.sort(np.random.uniform(0., 50, 1000))
lv_prediction_energy = lv_model.energy_model_train(lv_test_t)
lv_prediction_mse = lv_model.model_train(lv_test_t)

# q = ln(y), p = ln(x) 

# grand-truth
x, y = lotka_volterra(alpha=2/3, beta=4/3, delta=1., gamma=1., x0=0.9, y0=0.9, t_interval=[0,50], h=500)
plt.plot(x, y, 'o', c ="green", alpha=0.6, label='grand-truth')
# symplectic euler
plt.plot(np.exp(lv_model.z[:,1]), np.exp(lv_model.z[:,0]), c ="blue", alpha=0.6, label='symplectic-euler')
# NN with mse
plt.plot(np.exp(lv_prediction_mse[:,1]), np.exp(lv_prediction_mse[:,0]), c ="orange", alpha=0.6, label='NN with mse')
# NN with energy loss
plt.plot(np.exp(lv_prediction_energy[:,1]), np.exp(lv_prediction_energy[:,0]), c="red", alpha=0.6, label='NN with energy loss')
plt.legend(loc='upper right', bbox_to_anchor=(2.04, 1.036))

In [ ]:
plt.figure(figsize = (4, 4))
plt.xlabel('t', fontsize=17)
plt.ylabel('Energy', fontsize=17)

lv_energy = [lv_model.H(lv_T, lv_V, -0.105, -0.105) for i in np.linspace(0., 50., 40)]
lv_energy_new = lv_model.H(lv_T, lv_V, lv_prediction_energy[:,0], lv_prediction_energy[:,1])
lv_symplectic_new = lv_model.H(lv_T, lv_V, lv_model.z[:,0], lv_model.z[:,1])

plt.plot(np.linspace(0., 50.0, 40), lv_energy, 'o', c='green', alpha=0.6, label='expected energy')
plt.plot(lv_test_t, lv_energy_new, c='red', alpha=0.6, label='calculated energy')
plt.plot(lv_model.t, lv_symplectic_new, c='blue', alpha=0.6, label='symplectic-euler energy')
plt.legend(loc='upper right', bbox_to_anchor=(2.19, 1.036))